<a href="https://colab.research.google.com/github/ohashi-gnct/exp4e/blob/master/multimedia2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

まず、データを読み込むため自分のGoogleドライブをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ドライブ直下にpngファイルを置くと、

`/content/drive/MyDrive/hoge.png`と指定すればファイルが使える

In [11]:
import numpy as np
import cv2
from ipywidgets import interact
import matplotlib.pyplot as plt

import librosa
import librosa.display
import soundfile as sf
from IPython.display import display, Audio

## 0. 基礎知識





### 画像データのフーリエ解析



まずは中身を見てみる

`print()`すると、中身は`int`の配列になっていることがわかる

matplotplibで画像として`show`してみる

In [ ]:
# グレースケールで画像を読み込み
img = cv2.imread('/content/drive/MyDrive/lena_std.png',0)
print(img)
plt.gray()
plt.imshow(img)           
plt.show()

numpyの`numpy.fft.fft2()`でFFT（高速フーリエ変換）すると、

画像の周波数スペクトルが表示される

まんなかを中心として、離れるほど高周波

In [ ]:
img_fft = np.fft.fft2(img)
fshift = np.fft.fftshift(img_fft)
magnitude_spectrum = 20*np.log(np.abs(fshift))

print(magnitude_spectrum)
plt.gray()
plt.imshow(magnitude_spectrum) 
plt.xticks([])
plt.yticks([])          
plt.show()

FFTしたスペクトル`img_fft`をもとに、

画像を再構成してみる

もとの画像に戻った


In [ ]:
ifimage = np.fft.ifft2(img_fft) 
ifimage = ifimage.real

plt.imshow(ifimage)           
plt.show()

### 音声データのフーリエ解析

LibROSAにあるサンプル曲を読み込み、

波形とスペクトログラム（周波数スペクトルの時間変化）を出す

人間は高周波ほど小さな周波数の変化を認識できなくなるため、

高周波ほど荒いメル尺度でスペクトログラムを表示する

In [ ]:
def show_spectrogram(y, sr):
  S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
  log_S = librosa.power_to_db(S, ref=np.max)
  plt.figure(figsize=(10, 4))
  librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')
  plt.title('mel power spectrogram')
  plt.colorbar(format='%+02.0f dB')
  plt.show()

def show_wave_spectrogram_audio(y, sr):
  fig, [ax_wave, ax_spectrogram] = plt.subplots(2, 1, figsize=(12, 10))
  S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
  log_S = librosa.power_to_db(S, ref=np.max)
  librosa.display.waveplot(y, sr=sr, x_axis='time', ax=ax_wave)
  spectrogram = librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel', ax=ax_spectrogram)
  plt.show()
  display(Audio(y, rate=sr))

In [ ]:
y, sr = librosa.load('/content/drive/MyDrive/famipop3.mp3', mono=True)

show_wave_spectrogram_audio(y, sr)

## 1. 画像の周波数領域フィルタ（必須課題）

先ほどのスペクトルを見てみると、2次元になっている





周波数と聞くと1次元のイメージだが、

なぜ2次元で表示されているのだろう？

さまざまな画像を入力し、スペクトルの出かたを比べてほしい

http://www.ess.ic.kanagawa-it.ac.jp/app_images_j.html より画像引用


In [ ]:
def calc_spectrum(img):
  img_fft = np.fft.fft2(img)
  fshift = np.fft.fftshift(img_fft)
  return 20*np.log(np.abs(fshift))

# ここのファイル名を変える
img1 = cv2.imread('/content/drive/MyDrive/lena_std.png',0)
img2 = cv2.imread('/content/drive/MyDrive/Fishing-Boat.png',0)
img3 = cv2.imread('/content/drive/MyDrive/Pixel-ruler.png',0)


fig, [[ax1, ax2, ax3], [ax1_fft, ax2_fft, ax3_fft]] = plt.subplots(2, 3, figsize=(15, 8))

ax1.imshow(img1)
ax1_fft.imshow(calc_spectrum(img1))
ax1_fft.set_xticks([])
ax1_fft.set_yticks([])

ax2.imshow(img2)
ax2_fft.imshow(calc_spectrum(img2))
ax2_fft.set_xticks([])
ax2_fft.set_yticks([])

ax3.imshow(img3)
ax3_fft.imshow(calc_spectrum(img3))
ax3_fft.set_xticks([])
ax3_fft.set_yticks([])

plt.show()

**↑の図を3.1の結果として授業中に提出**

スペクトルに対して操作を行うことで、周波数領域から画像を処理できる。

ローパスフィルターをかけてみる。

スペクトルの中心部だけ残すようにスペクトルをマスクすると、

ローパスフィルターとして機能する

再構成した画像は、帽子などの輪郭のまわりに波のような模様が出てしまった

この波は何だろうか？



低周波の領域だけで再構成しても、

画像の（見た目の）特徴をある程度残しているように見える

→高周波の情報を減らすことで、見た目をそのままに

画像のデータ量を減らすことができる？

In [ ]:
# ここのRを大きくすればするほど、高周波まで通す
def mask_to_fshift(fshift, R):
  size = fshift.shape
  mask = np.zeros(size)
  length = size[0]
  centery = size[0]/2
  for x in range(0,length):
    for y in range(0,length):
      if (x- centery)**2 +(y- centery)**2 <R**2:
        mask[x,y]=1
  return fshift*mask

plt.gray()

def LPF(R=30):
  fig, [ax_fft, ax_image] = plt.subplots(1, 2, figsize=(12, 8))
  fshift_mask = mask_to_fshift(fshift.copy(), R)
  magnitude_spectrum = 20*np.log(np.abs(fshift_mask))
  ax_fft.imshow(magnitude_spectrum)
  ax_fft.set_xticks([])
  ax_fft.set_yticks([])

  fft_mask = np.fft.fftshift(fshift_mask) 
  ax_image.imshow(np.fft.ifft2(fft_mask).real)
  plt.show()

interact(LPF, R=(0, 300))

## 2. 画像の空間領域フィルタと周波数（必須課題）

2年生の実験で画像のフィルタリングを学習した。

画像に対して、例えば5x5のフィルタを畳み込む

画像の平滑化フィルタとして、

移動平均フィルタ`kernel_movingaverage`と、

ガウシアンフィルタ`kernel_gaussian`を用意した。

見た目ではどのように変化したか？

In [ ]:
# 5x5 のフィルタを使っている。好きなフィルタに変えてみてもよい
kernel_original = np.array([[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0],
                            [0, 0, 1, 0, 0],
                            [0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]])

kernel_movingaverage = np.array([[1, 1, 1, 1, 1],
                                 [1, 1, 1, 1, 1],
                                 [1, 1, 1, 1, 1],
                                 [1, 1, 1, 1, 1],
                                 [1, 1, 1, 1, 1]]) / 25

kernel_gaussian = np.array([[1,  4,  6,  4, 1],
                            [4, 16, 24, 16, 4],
                            [6, 24, 36, 24, 6],
                            [4, 16, 24, 16, 4],
                            [1,  4,  6,  4, 1]]) / 256

fig, [ax_img_original, ax_img_movingaverage, ax_img_gaussian] = plt.subplots(1, 3, figsize=(15, 8))

img = cv2.imread('/content/drive/MyDrive/lena_std.png',0)

img_original = cv2.filter2D(img, -1, kernel_original)
ax_img_original.imshow(img_original)
ax_img_original.set_title("Original")

img_movingaverage = cv2.filter2D(img, -1, kernel_movingaverage)
ax_img_movingaverage.imshow(img_movingaverage)
ax_img_movingaverage.set_title("Moving Average")

img_gaussian = cv2.filter2D(img, -1, kernel_gaussian)
ax_img_gaussian.imshow(img_gaussian)
ax_img_gaussian.set_title("Gaussian")

plt.show()

#### 移動平均フィルタ

フィルタを3x3, 5x5, 7x7…と大きくしてアニメーションを作成する

フィルタが大きいほど平滑化の効果は強くなる

周波数領域で見ると、どんな役割をしているか？



In [23]:
def show_spectrum(img):
  img_fft = np.fft.fft2(img)
  fshift = np.fft.fftshift(img_fft)
  return 20*np.log(np.abs(fshift))
  ax_fft.set_xticks([])
  ax_fft.set_yticks([])

In [ ]:
def movingaverage(i=5):
  fig, [ax_fft, ax_image] = plt.subplots(1, 2, figsize=(12, 8))
  img_movingaverage = cv2.blur(img, (i, i))
  ax_image.imshow(img_movingaverage)   
  ax_fft.imshow(show_spectrum(img_movingaverage))

interact(movingaverage, i=(1, 49, 2))

#### ガウシアンフィルタ

ガウシアンフィルタでは、周りの画素を一律に平均するのではなく、

注目している画素の近くに重み付けしてある

$$ f(x, y)=\frac{1}{2\pi\sigma^2} \exp({-\frac{x^2+y^2}{2\sigma^2}})$$

フィルタを3x3, 5x5, 7x7…と大きくし、それに合わせて標準偏差 $ \sigma $ を大きくする

標準偏差が大きいほど平滑化の効果は強くなる

周波数領域で見ると、どんな役割をしているか？

In [ ]:
def gaussian(i=5):
  fig, [ax_fft, ax_image] = plt.subplots(1, 2, figsize=(12, 8))
  img_gaussian = cv2.GaussianBlur(img, (i, i), i/2)
  ax_image.imshow(img_gaussian)   
  ax_fft.imshow(show_spectrum(img_gaussian))

interact(gaussian, i=(1, 49, 2))

**↑の図を3.2の結果として授業中に提出**

ぼかしのかけ方は適当に調節したほうがよい


## 3. 音声の周波数領域フィルタ（必須課題）

高周波ノイズが加えられた音声を用意する

ノイズの入った音声のスペクトログラムを見ると、

高周波に強いエネルギーがある

In [ ]:
# ここのファイル名を変える。自分で高周波ノイズ入りの音声を作っても良い
y_voice, sr_voice = librosa.load('/content/drive/MyDrive/voice_with_noise_100.wav')
show_wave_spectrogram_audio(y_voice, sr_voice)

FFTを行い、ある周波数以上をカットすると、ノイズがなくなった

In [ ]:
n_fft = 2048
D = librosa.stft(y_voice, n_fft=n_fft)
print(D.shape)
# ここの96を変えると、どの周波数まで通すかが変わる
# この数値はFFTした配列の話であって、Hzではない。
D[96:] = 0
y_voice_lowpass = librosa.istft(D)
show_wave_spectrogram_audio(y_voice_lowpass, sr_voice)

次は音楽を使ってみる

まずは読み込み

In [ ]:
# ここのファイル名を好きな音楽に変える。mono=Trueでモノラルになる
y_tropical, sr_tropical = librosa.load('/content/drive/MyDrive/famipop3.mp3', mono=True)
print(y_tropical.shape)
print(sr_tropical)
# 曲が長すぎるので適当に切り出す
# サンプリングレートが44100Hzであれば、1秒間に44100サンプルあるということになる
y_tropical = y_tropical[:500000]
show_wave_spectrogram_audio(y_tropical, sr_tropical)

ローパスフィルターやハイパスフィルターを試す

聴覚上の変化はどうなる？スペクトログラム上では？

In [ ]:
n_fft = 2048
D = librosa.stft(y_tropical, n_fft=n_fft)
flag = 0
freq_pass = 10

# このフィルタは自由に変えて良い
# ハイパスとローパスを交互にかけると面白いかなと思ってこうしてあるだけ
for i in range(len(D[0])):
  if flag == 0:
    D[int(freq_pass):, i] = 0
    freq_pass *= 1.00 + (i * 0.0001)
    if freq_pass > 10000:
      flag = 1
      freq_pass = 20
  elif flag == 1:
    D[:int(freq_pass), i] = 0
    freq_pass *= 1.00 + (i * 0.0001)
    if freq_pass > 2000:
      flag = 0
      freq_pass = 20

y_tropical_lowpass = librosa.istft(D)
show_wave_spectrogram_audio(y_tropical_lowpass, sr_tropical)

**↑の波形を3.3の結果として授業中に提出**

フィルタは適当に用意したものなので、

自分の気に入る形のフィルタを作って提出すること

もちろん、ただのローパスフィルタでもよい

↓フィルタの例

In [ ]:
n_fft = 2048
D = librosa.stft(y_tropical, n_fft=n_fft)
D[96:] = 0

y_tropical_lowpass = librosa.istft(D)
show_wave_spectrogram_audio(y_tropical_lowpass, sr_tropical)

## 4. 音声の時間領域フィルタと周波数（発展課題）

$ n $ 点の平均をとるような移動平均フィルタを設計し、

音楽に畳み込む

聞こえ方はどう変化するか？周波数上ではどうか？

In [ ]:
# nを変えるとフィルタの長さが変わる
# とうぜん長いほど平滑化の効果は大きい
n = 30
filter_movingaverage = np.ones(n)/n
plt.plot(filter_movingaverage)
plt.xlim(0, n-1) 
plt.ylim(0, 0.1)
plt.show()

In [ ]:
y_tropical_filtered = np.convolve(y_tropical, filter_movingaverage)
show_wave_spectrogram_audio(y_tropical_filtered, sr_tropical)

今度は
$ A\exp{-t/\tau} $
の形で減衰する

指数平滑フィルタを設計し、畳み込む

移動平均フィルタと違いはあるか？

In [ ]:
# nを変えるとフィルタの長さが変わる
n = 30
filter_exp = np.exp(-np.arange(n)/(n/3))
filter_exp /= np.sum(filter_exp)
plt.plot(filter_exp)
plt.xlim(0, n-1)
plt.ylim(0, 0.1)
plt.show()

In [ ]:
y_tropical_filtered = np.convolve(y_tropical, filter_exp)
show_wave_spectrogram_audio(y_tropical_filtered, sr_tropical)

## 5. サンプリングと波形の復元（発展課題）

まず、正弦波を荒くサンプリングする

In [ ]:
# 周期あたり何点サンプリングするか。ここが2未満だとどうなるか？
sampling_per_period = 5

sampling_period = int(100 / sampling_per_period)
x = np.linspace(0, 3, 301)
y = np.cos(x*np.pi*2)
x_sampling = x[::sampling_period]
y_sampling = y[::sampling_period]
print(y_sampling)
plt.plot(x, y)
plt.scatter(x_sampling, y_sampling)
plt.legend()
plt.show()

サンプリングした点だけ出すと

In [ ]:
plt.scatter(x_sampling, y_sampling)

これらの点をパルス列とみなして

sinc関数 $ \mathrm{sinc}(x) = \sin{x}/x $ を畳み込むことで、

波形の復元ができる

sinc関数はこんな形の関数


In [ ]:
x_temp = np.linspace(-6, 6, 1201)
plt.plot(x_temp, np.sinc(x_temp))

sinc関数を畳み込む

よく見ると、サンプリングした点は1つのsinc関数を除いて

すべて値が0になっている

サンプリングした点どうしを滑らかにつなぐような

線になっていることがわかる

In [ ]:
sincs = []

for y_temp, x_temp in zip(y_sampling, x_sampling):
  sincs.append(y_temp * np.sinc(100 / sampling_period * (x_temp - x)))

for sinc in sincs:
  plt.plot(x, sinc, color = "green")

sum_sinc = sum(sincs)
plt.plot(x, sum_sinc, color="blue", lw=3)

plt.scatter(x_sampling, y_sampling, color="red")

1周期あたりのサンプル数や、初期位相を変えて

波形の復元を試してみる。

とくに、1周期のサンプル数を2点未満にするとどうか？